In [18]:
##################################### IMPORT ######################################
# package listing                                                                 #
###################################################################################
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
import csv as csv
import requests
import pandas as pd
import time
import numpy as np
from sklearn.cluster import KMeans
import mechanicalsoup
pd.options.mode.chained_assignment = None

In [19]:
def periodic_work(interval):
    while True:
        #################################### GET DATA #####################################
        # Get all available Roster Update data                                            #
        ###################################################################################

        # initialize player current price & profit
        df=None
        code=[] #player ID
        name=[] #player name
        rating=[] #player rating
        buy=[] #card buy now
        sell=[] #card sell now
        profit=[] #card profit

        url = 'https://mlb19.theshownation.com/community_market'
        prefix = 'https://mlb19.theshownation.com/community_market?page='
        suffix = '&type_id=0'
        page1 = requests.get(url)
        soup1 = BeautifulSoup(page1.text,'html.parser')

        # index
        max_ind = soup1.find('h3').get_text().split(' ')[6]

        for i in range(int(max_ind)):
            page1 = requests.get(prefix + str(i) + suffix)
            soup2 = BeautifulSoup(page1.text,'html.parser')

            # soup
            list_tags = soup2.find('div').find_all('td')
            a_tags = soup2.select("a[href*=community_market]")

            a_tags.pop(len(a_tags)-1)

            for l in range(0,2):
                a_tags.pop(l)

            # ids
            for j in range(0,int(len(a_tags)/2-1)):
                code.append(a_tags[j*2+1].get('href').split("/")[3])

            # names, prices
            for k in range(0,int(len(list_tags)/9-1)):
                name.append(list_tags[k*9+1].get_text().strip())
                rating.append(int(list_tags[k*9+2].get_text().strip()))
                buy.append(int(list_tags[k*9+3].get_text()))
                sell.append(int(list_tags[k*9+4].get_text()))

        maxprofit = lambda x,y: 0.9*int(x)-int(y)
        profit = map(maxprofit,buy,sell)

        df = pd.DataFrame(zip(name,rating,buy,sell,profit,code),columns=['Name','Rating','Buy','Sell','Profit','Code'])

        if name:
            name.clear(), rating.clear(), buy.clear(), sell.clear(), code.clear()

        ################################### FILTERING #####################################
        # in order to cut down the number of records, apply filters here                  #
        # store original results in copy                                                  #
        ###################################################################################

        #FILTER 
        #in order to cut down the number of records, apply filters here
        #store original results in copy
        orig=df

        #filter, sort
        df=df[(df['Rating']>70) & (df['Profit']>1000)].sort_values('Profit',ascending=False).reset_index(drop=True)

        ############################### HISTORICAL PRICES #################################
        # using df from above, ping cards by code and find latest buy/sell prices,        #
        ###################################################################################

        # initilaize historical price
        datesf=None
        hprice=[] #historical price df
        hdt=[] #historical datetime df
        hdate=[] #historical date
        codef=[]

        player_url = 'https://mlb19.theshownation.com/community_market/listings/'

        for j in range(0,int(len(df)-1)): ## use this logic to assemble the entire card list

            suffix = df['Code'][j]
            player = requests.get(player_url+suffix)
            player_soup = BeautifulSoup(player.text,'html.parser')

            try:
                record_tags = player_soup.find('table', id='table-completed-orders').find_all('td')
            except:
                continue
            ##record_tags[2n: 0_to_50] = prices
            ##record_tags[2n+1: 0_to_50] = times

            #experiment with expanding this - the site suggests only 50 prices are available but 
            #it seems like there may be no limit in terms of price
            for i in range(0,50):
                try:
                    #strip outside newlines, split by interior newlines and take str index 2 (the amount)
                    #apply translation replacing comma w/ null to put string number in form python can 
                    #interpret as int in order to utilize 
                    hprice.append(int(record_tags[2*i].get_text().strip().split('\n')[2].translate(str.maketrans({',':''}))))

                    #create datetime object for each sale, this will let us do math to dates/times
                    #have to remove PDT time zone reference because datetime can't parse this correctly
                    #instead, just add 2 hours to make up for the time difference for easy review
                    hdt.append(datetime.strptime(record_tags[2*i +1].get_text().replace(' PDT',''), '%m/%d/%Y %I:%M%p') + timedelta(hours=2))

                    #append readable date 
                    hdate.append(hdt[i].strftime('%m/%d/%Y %I:%M%p'))
                    codef.append(suffix)
                except:
                    continue

            datesf = pd.DataFrame(zip(codef,hprice,hdate,hdt),columns=['Code','Price','Date','DateTime'])

            if hprice:
                hprice.clear, hdate.clear, hdt.clear

        ################################### CLUSTERING ####################################
        # we have 2 clusters, quick buy & order buy, simple way to detect which is which: #
        # k-means clustering. i do that here                                              #
        ###################################################################################

        datesf2=None
        datesf2=datesf
        unique_code = []

        # need a unique UNSORTED list of codes
        data = datesf2['Code']
        seen = set()

        for x in datesf2['Code']:
            if x not in seen:
                unique_code.append(x)
                seen.add(x)

        for code in unique_code:
            # create kmeans object
            kmeans = KMeans(n_clusters=2)

            # fit kmeans object to data
            kmeans.fit(datesf2.loc[datesf2['Code']==code][['Price']])

            # make sure that the cluster associated w/ quick buy is always = 1
            idx = np.argsort(kmeans.cluster_centers_.sum(axis=1))
            lut = np.zeros_like(idx)
            lut[idx] = np.arange(2)

            # append clusters to indicate whether something is a quickbuy or an order
            y_km = lut[kmeans.labels_]

            datesf2.loc[datesf2['Code']==code,'QB'] = y_km

        ################################### LIQUIDITY #####################################
        # we have 2 clusters, quick buy & order buy, simple way to detect which is which: #
        # k-means clustering. i do that here                                              #
        ###################################################################################

        # caveat: observing a small sample (perhaps should expand view?) of a complex stoch. 
        # process and trying to apply simple models to it. metrics are simple
        # goal: improving the capital return & efficiency in terms of time spent adjusting buys & sells

        # QB is similar to a poisson process: generally reasonable to assume inter-arrival times 
        # are consistent buy-to-buy given that the process which generates events should be IID
        # really though, this model is useful as it gives us a way to judge liquidity on QB mkt

        # we can consider metrics for evaluating liqudity: 
        ### how many QBs out of last 100 sales? 
        #### an ideal flip candidate would have a higher QB ratio
        ### what is QB inter-arrival time (average) over last 100 sales?
        #### an ideal flip candidate would have a smaller inter-arrival value for QB
        #### using this information, we can potentially forecast future arrival for QBs to anticipate
        #### QB behavior and be positioned

        basef=None ## base data for liq. metrics
        metrics=None ## hold the metrics
        codes=[]
        lam=[]
        qbtt=[]
        diff=[]

        # collect base data & reset index
        basef = datesf2[datesf2['QB']==1].reset_index(drop=True)
        ind = 0

        # determine time between QBs
        for code in unique_code:

            ## initialize the list for every Code
            if diff:
                diff.clear()
            else:
                diff=[]

            # apply diff down the ladder of QB times until the last 
            for i in range(ind,ind + len(basef.loc[basef['Code']==code])-1): 
                tdiff = (basef['DateTime'][i] - basef['DateTime'][i+1]).total_seconds()/60
                diff.append(int(tdiff))

            ind = ind + len(basef.loc[basef['Code']==code])

            # insert a 0 for the final entry of each code
            diff.insert(len(diff),0)

            # push diff list back to the base data by-code
            basef.loc[basef['Code']==code,'Diff'] = diff

            ## using above metrics, calculate some additional statistics:
            # average inter-arrival time (Poisson lambda)
            lsum = sum(diff)/len(diff)
            lam.append(lsum)

            # proportion of quick-buys to total orders ~ relative frequency 
            qsum = sum(basef.loc[basef['Code']==code]['QB'])/100
            qbtt.append(qsum)

        metrics = pd.DataFrame(zip(unique_code, lam, qbtt), columns=['Code','Lambda','QBR'])

        # inner join back w/ original 
        df_inner = pd.merge(df, metrics, on='Code', how='inner')

        base_url = 'https://mlb19.theshownation.com/community_market/listings/'

        df_inner["URL"] = base_url + df_inner['Code']

        #df_inner[(df_inner["Sell"]>0) & (df_inner["Lambda"]>0)].sort_values('Lambda')
        df_inner[(df_inner["Sell"]>0) & (df_inner['Lambda']>0) & (df_inner['Lambda']<60) & (df_inner['Profit']>1000)].sort_values('Lambda').to_csv("buys.csv")
        
        time.sleep(interval)

In [ ]:
periodic_work(150)